In [1]:
import numpy as np
import matplotlib.pyplot as plt
import struct
import math
from statistics import mean, stdev

import qiskit
from qiskit import *

import pennylane as qml
from pennylane import numpy as pnp

import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

In [2]:
# float to binary 
def float_to_bin_real(num):
    return format(struct.unpack('!I', struct.pack('!f', num))[0], '032b')
def float_to_bin_imag(num):
    return format(struct.unpack('!I', struct.pack('!f', num))[0], '032b')

# binary to float
def bin_to_float_real(binary):
    return struct.unpack('!f',struct.pack('!I', int(binary, 2)))[0]
def bin_to_float_imag(binary):
    return struct.unpack('!f',struct.pack('!I', int(binary, 2)))[0]

# reverse a string (for the measurement step readout)
def reverse(string):
    string = string[::-1]
    return string

In [3]:
def generate_img(n):

    sky_image = np.zeros((n, n), dtype='complex64')
    mask = np.zeros((n, n), dtype='bool')
    stars = np.random.randint(1, 7) 
    radius = 1 

    for i in range(0, stars):
        index_i = np.random.randint(0, n-radius)
        index_j = np.random.randint(0, n-radius)
        for j in range(0, radius):
            for k in range(0, radius):
                sky_image[index_i+j][index_j+k] = complex(np.random.rand()*100, np.random.rand()*100)
                mask[index_i+j][index_j+k] = True
            
    sky_image.real += np.random.randn(n,n)/100
    sky_image.imag += np.random.randn(n,n)/100
    
    visibilities = np.fft.fft2(sky_image).astype('complex64')
    
    return sky_image, visibilities

In [4]:
def binary_encode(qc, binary, off_set):
    
    for i in range(0, len(binary)):
        qc.reset(off_set+i) 

        if binary[i]=='1':
            qc.x(off_set+i) 
    
    off_set += len(binary)

    return off_set

In [5]:
def binary(img):
    
    number_of_pixels = img.shape[0]*img.shape[1]
    number_of_bits_real = 32
    number_of_bits_imag = 32
    number_of_bits = number_of_bits_real + number_of_bits_imag
    number_of_qbits = number_of_pixels*number_of_bits
    off_set = 0

    qc = QuantumCircuit(number_of_qbits)
    for i in range(0, img.shape[0]):
        for j in range(0, img.shape[1]):
                binary_real = float_to_bin_real(img[i, j].real)
                binary_imag = float_to_bin_imag(img[i, j].imag)
                binary = binary_real+binary_imag
                off_set = binary_encode(qc, binary, off_set)
                
    qc.measure_all()
    backend = Aer.get_backend('aer_simulator') 
    job = backend.run(qc, shots=1, memory=True) 
    output = job.result().get_memory()[0]
    out = reverse(output)
    
    chunks_real = []
    chunks_imag = []
    for i in range(0, number_of_pixels):
        chunks_real.append(out[number_of_bits*i:(number_of_bits*i)+number_of_bits_real]) 
        chunks_imag.append(out[(number_of_bits*i)+number_of_bits_imag:(number_of_bits*i)+number_of_bits_imag+number_of_bits_imag]) 

    readout = []
    for i in range(0, len(chunks_real)):
        readout.append(complex(bin_to_float_real(chunks_real[i]), bin_to_float_imag(chunks_imag[i]))) 

    readout = np.array(readout).reshape(img.shape[0] , img.shape[1]).astype('complex64')
    readout = np.fft.ifft2(readout).astype('complex64') #IFT
    
    return readout

In [6]:
def MSE(img1, img2):
    diff_real = (img1.real - img2.real)
    diff_imag = (img1.imag - img2.imag)
    diff_sum = diff_real + diff_imag
    
    mse = (diff_sum**2).mean()
    
    return mse

In [7]:
img_sizes = [2, 3, 4, 5, 6, 7, 8, 9, 10]
basis_results = []
amplitude_results = []

for i in tqdm(range(0, len(img_sizes))):
    
    iterations = 100
    img_size = img_sizes[i]
    binary_MSE = []
    amplitude_MSE = []

    for j in range(0, iterations):

        initial, visibilities = generate_img(img_size) 

        # BINARY
        binary_readout = binary(visibilities)

        # AMPLITUDE
        n_n = img_size*img_size
        inp = visibilities.flatten()
        norm = qml.math.sum(qml.math.abs(inp) ** 2)
        number_of_qbits = math.ceil(math.log2(n_n))
        wires = range(number_of_qbits)
        amp_dev = qml.device('default.qubit', wires)
        @qml.qnode(amp_dev)
        def amp_encoding(data):
            qml.AmplitudeEmbedding(data, wires, normalize=True, pad_with=0.)
            return qml.state()
        readout = amp_encoding(inp).astype('complex64')
        readout.numpy()
        readout = readout*qml.math.sqrt(norm) 
        readout = readout[:n_n] 
        readout = np.array(readout).reshape(img_size , img_size)
        amplitude_readout = np.fft.ifft2(readout).astype('complex64')

        binary_mse = MSE(initial, binary_readout)
        amplitude_mse = MSE(initial, amplitude_readout)

        binary_MSE.append(binary_mse)
        amplitude_MSE.append(amplitude_mse)

    binary_mean = mean(binary_MSE)
    binary_std = np.std(binary_MSE)
    amplitude_mean = mean(amplitude_MSE)
    amplitude_std = np.std(amplitude_MSE)
    
    basis_results.append((binary_mean, binary_std))
    amplitude_results.append((amplitude_mean, amplitude_std))

print('Basis results: '+ '\033[1m' + str(basis_results) + '\033[0m')
print()
print('Amplitude results: '+ '\033[1m' + str(amplitude_results) + '\033[0m')

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [11:16<00:00, 75.13s/it]

Basis results: [(9.577728e-13, 8.8037216e-13), (9.689503e-13, 7.8417513e-13), (6.789412e-13, 5.75877e-13), (5.028152e-13, 3.5090526e-13), (3.5514687e-13, 2.3066285e-13), (2.5876482e-13, 1.5975609e-13), (2.3207193e-13, 1.4038723e-13), (1.7368042e-13, 9.595658e-14), (1.4768637e-13, 9.1425694e-14)]

Amplitude results: [(6.4084437e-12, 1.4379955e-11), (7.8579174e-12, 1.3219338e-11), (3.3644146e-12, 4.4576252e-12), (3.8621203e-12, 5.7174578e-12), (3.8226614e-12, 5.5462436e-12), (2.0644207e-12, 2.9238385e-12), (9.216979e-13, 1.0753443e-12), (1.1774036e-12, 1.6761479e-12), (1.5857309e-12, 2.1842376e-12)]


In [8]:
basis_tmp = []
for i in range(0, len(basis_results)):
    basis_tmp.append(basis_results[i][0])
basis_absolute_mean = mean(basis_tmp)

amplitude_tmp = []
for i in range(0, len(amplitude_results)):
    amplitude_tmp.append(amplitude_results[i][0])
amplitude_absolute_mean = mean(amplitude_tmp)

print('Basis absolute mean: '+ '\033[1m' + str(basis_absolute_mean) + '\033[0m')
print()
print('Amplitude absolute mean: '+ '\033[1m' + str(amplitude_absolute_mean) + '\033[0m')

Basis absolute mean: 4.750922e-13

Amplitude absolute mean: 3.4516457e-12
